In [1]:
from persona_generation_prompt import PROMPT

In [2]:
import json

with open("../knowledge_base/mental_health_subjects.json", "r") as f:
    mental_health_subjects = json.load(f)

print(mental_health_subjects)

{'سلامت معنوی در موقعیت کاهش استقلال': [{'seconday_subject': 'ربوبیت باوری', 'aspect': 'شناخت', 'relational_reference': 'خدا', 'description': 'سالمند دارای سلامت معنوی به رب بودن پروردگار در هستی باور دارد. این باور که رب و پرورش دهنده همه یکی است و در طول همه مسولیتها خداوند قرار میگیرد پذیرش سالمند را نسبت به تغییر نقش و جایگاه فعلی اش افزایش می دهد.'}, {'seconday_subject': 'وسعت بخشی معنای استقلال', 'aspect': 'شناخت', 'relational_reference': 'خود', 'description': 'وقتی سالمند استقلال را به معنای تسلط بر همه امور و مستقل عمل کردن مانند دوره جوانی بداند پذیرش شرایط جدید که محصول کاهش توانمندیها و تغییر اغتضاعات زندگی است برای وی دشوار خواهد شد. اگر سالمند پذیرا بودن نسبت به حمایت دیگران را به کاهش استقلال تعبیر نکند و استقلال برای او به مفاهیم فرامادی هم گسترش یافته باشد، در واقع سالمند می تواند از این رنج با سلامت معنوی عبور کند به عنوان مثال یک سالمند ممکن است عدم توانایی برای رانندگی را کاهش استقلال تلقی کند و از آن رنج ببرد و سالمند دارای سلامت معنوی استقلال در اندیشه و صاحب نظر ب

In [3]:
prompt = PROMPT.replace("{mental_health_subjects}",
                        json.dumps(mental_health_subjects, ensure_ascii=False, indent=2))
neg_persona_instructions = "- شخصیت را با ویژگی‌های منفی بسازید طوری که این ویژگی‌ها در تمام جنبه‌های شخصیت مشهود باشد."
pos_persona_instructions = "- شخصیت را با ویژگی‌های مثبت بسازید طوری که این ویژگی‌ها در تمام جنبه‌های شخصیت مشهود باشد."

neg_persona_prompt = prompt.replace("{additional_instructions}", neg_persona_instructions)
pos_persona_prompt = prompt.replace("{additional_instructions}", pos_persona_instructions)

print(pos_persona_prompt)

شما باید تعدادی شخصیت فرضی سالمند ایرانی تولید کنید که نمایانگر تنوع فرهنگی، جغرافیایی، و اجتماعی ایران باشند.

قوانین:
- همه شخصیت‌ها باید سالمند (بالای ۶۵ سال) باشند.
- ترکیب شخصیت‌ها باید شامل تنوع در منطقه زندگی (شمال، جنوب، شرق، غرب، مرکز)، قومیت (فارس، ترک، کرد، لر، بلوچ، عرب و ...)، مذهب، وضعیت مالی، سطح تحصیلات، و پیشه باشد.
- شخصیت‌ها باید بر اساس بافت فرهنگی ایران و واقعیات اجتماعی ساخته شوند.
- واکنش‌ها و نگرش‌ها لازم نیست همیشه "صحیح" یا "اخلاقی" باشند؛ می‌توانند متأثر از تجربه، فرهنگ، باورها یا محدودیت‌های فرد باشند.
- صفات شخصیتی باید چندبُعدی و متناسب با پیشینه فرد باشند.
- در قسمت توضیحات (background) یک توصیف کوتاه ولی جزئیات‌دار از زندگی فرد، تجربیات، ارزش‌ها، و سبک زندگی او بیاورید.
- برای هر یک از ۹ حیطه "سلامت معنوی" وضعیت شخص را در ۲ الی ۳ جمله بیان کن."
- از فرمت JSON زیر استفاده کن.
- برای هر کلید، از مقادیر واقعی و سازگار با فرهنگ ایران استفاده کن.
- شخصیت را با ویژگی‌های مثبت بسازید طوری که این ویژگی‌ها در تمام جنبه‌های شخصیت مشهود باشد.

توضیحات حوزه‌های سلام

In [4]:
from openai import OpenAI
import os

AVALAI_BASE_URL = "https://api.avalai.ai/v1"

client = OpenAI(
    api_key=os.getenv("AVALAI_API_KEY"),
    base_url=AVALAI_BASE_URL,
)

In [5]:
model_to_use = "gpt-4o"

TOP_P = 0.9
TEMPERATURE = 0.7

In [6]:
from typing import List
from openai.types.chat import (
    ChatCompletionAssistantMessageParam,
    ChatCompletionMessageParam,
    ChatCompletionSystemMessageParam,
    ChatCompletionUserMessageParam,
)
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage

def messages(polarity: str):
    return [
    ChatCompletionSystemMessageParam(role="system", content=pos_persona_prompt if polarity == "pos" else neg_persona_prompt),
    ChatCompletionUserMessageParam(role="user", content="Generate 10 personas. Only give the JSON array with no extra text or formatting. Don't wrap the array in markdown formatting.")
]


In [7]:
# OpenAI-compatible client (Avalai)
AVALAI_BASE_URL = os.getenv("AVALAI_BASE_URL", "https://api.avalai.ir/v1")
AVALAI_MODEL = os.getenv("AVALAI_MODEL", "gpt-4o")

TEMPERATURE = 0.7
TOP_P = 0.9
PRESENCE_PENALTY = 0.3
FREQUENCY_PENALTY = 0.4

client = OpenAI(
    api_key=os.getenv("AVALAI_API_KEY"),
    base_url=AVALAI_BASE_URL,
)

class LLMCaller:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model

    def _role(self, m):
        return "assistant" if m.type == "ai" else "user" if m.type == "human" else "system"
    
    def _text(self, content) -> str:
        if isinstance(content, str):
            return content
        if isinstance(content, list):
            parts = []
            for p in content:
                if isinstance(p, dict):
                    parts.append(p.get("text") or p.get("content") or "")
                else:
                    parts.append(str(p))
            return "\n".join(x for x in parts if x)
        return str(content)

    def _build_payload(self, messages: List[BaseMessage]) -> List[ChatCompletionMessageParam]:
        payload = []
        for m in messages:
            if isinstance(m, SystemMessage):
                payload.append(ChatCompletionSystemMessageParam(
                    role="system",
                    content=self._text(m),
                ))
            elif isinstance(m, HumanMessage):
                payload.append(ChatCompletionUserMessageParam(
                    role="user",
                    content=self._text(m),
                ))
            elif isinstance(m, AIMessage):
                payload.append(ChatCompletionAssistantMessageParam(
                    role="assistant",
                    content=self._text(m),
                ))
        return payload

    def generate(self, messages, model: str | None = None):
        model_to_use = model or self.model
        resp = self.client.chat.completions.create(
            model=model_to_use,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            # presence_penalty=PRESENCE_PENALTY,
            # frequency_penalty=FREQUENCY_PENALTY,
            messages=messages,
        )
        return resp

llm = LLMCaller(client, AVALAI_MODEL)
print(f"Avalai client ready: {AVALAI_MODEL} @ {AVALAI_BASE_URL}")


Avalai client ready: gpt-4o @ https://api.avalai.ir/v1


In [12]:
resp = llm.generate(messages("neg"), model=model_to_use)

In [13]:
text = resp.choices[0].message.content

In [14]:
text

'[\n {\n "id": 1,\n "age": 70,\n "gender": "M",\n "level_of_education": "بی\u200cسواد",\n "occupation": "کشاورز",\n "financial_status": "ضعیف",\n "marital_status": "بیوه",\n "personality_traits": ["خودخواه", "لجوج", "بدبین"],\n "background": "اهل سیستان و بلوچستان، بلوچ، کشاورز ساده\u200cای که زمین\u200cهایش به دلیل خشکسالی از دست رفته\u200cاند. در طول زندگی هیچ\u200cوقت به تحصیلات علاقه نداشت و بیشتر وقتش را در مزرعه سپری می\u200cکرد. بعد از فوت همسرش، منزوی و بدبین شده و با فرزندانش رابطه سردی دارد.",\n "religion": "مسلمان سنی",\n "spiritual_health_loss_of_independence": "به دلیل کاهش استقلال، احساس ضعف می\u200cکند و کمک دیگران را نوعی تحقیر می\u200cداند.",\n "spiritual_health_loss_of_social_activity": "بعد از از دست دادن مزرعه، ارتباطات اجتماعی\u200cاش به شدت کاهش یافته و حاضر به تغییر سبک کنشگری نیست.",\n "spiritual_health_physical_health_and_sexual_issues": "افسوس زیادی برای جوانی و سلامت جسمی از دست رفته\u200cاش دارد و پذیرش تغییرات جسمی برایش دشوار است.",\n "spiritual_health_los

In [15]:
with open("personas2.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(json.loads(text), ensure_ascii=False))